In [43]:
import pandas as pd
import datetime

def missing_values(csv):
    df = pd.read_csv(csv)
    # grouped_df = df.groupby(["city"])
    return df[df.isna().any(axis=1)]



How many missing values we have and where


In [44]:
response = missing_values("../data/processed/weather_energy_data.csv")
print(response)
print(f"Missing Value Count: {len(response)}")

           date     city       state   TMAX  TMIN  \
771  2025-08-03  Phoenix     Arizona  112.0  84.0   
772  2025-08-03  Seattle  Washington  150.0   NaN   

                    respondent-name  timezone    value-units  Demand  \
771  Arizona Public Service Company  Mountain  megawatthours     NaN   
772              Seattle City Light   Pacific  megawatthours     NaN   

     Net generation  
771         78982.0  
772          7357.0  
Missing Value Count: 2


Any outliers (temperatures above 130°F or below -50°F, negative energy consumption)


In [45]:
df = pd.read_csv("../data/processed/weather_energy_data.csv")
df.head(1)

,date,city,state,TMAX,TMIN,respondent-name,timezone,value-units,Demand,Net generation
0,2025-03-01,Chicago,Illinois,34.0,22.0,"PJM Interconnection, LLC",Eastern,megawatthours,2110785.0,2206107.0


In [46]:
#filter weather report and let it return specific columns
filtered_df = df[(df["TMAX"]>130.0) | (df["TMIN"]<-50.0)]
bad_weather = filtered_df[["date", "city", "state", "TMAX", "TMIN"]]

# display result
# print(bad_weather)

In [47]:
#filter energy and return specific column
bad_energy = df[df["Demand"]<0][["date", "city", "state", "Demand", "value-units", "Net generation"]]

# display result
# print(bad_energy)

In [48]:
if not bad_weather.empty  or not bad_energy.empty:
    print(pd.concat([bad_weather, bad_energy], axis=0, ignore_index=True))
else:
    print(f"No outliers detected")


         date     city       state   TMAX  TMIN  Demand value-units  \
0  2025-08-03  Houston       Texas   98.0 -78.0     NaN         NaN   
1  2025-08-03  Seattle  Washington  150.0   NaN     NaN         NaN   

   Net generation  
0             NaN  
1             NaN  


reporting city freshness

In [49]:
import pandas as pd
import datetime

def report_city_freshness(csv_path, date_col="date", city_col="city"):
    # 1) Load and normalize
    df = pd.read_csv(csv_path) #header=None,
                    #  names=[date_col, "datatype", "station", "attrs", "value", city_col, "state"])
    # e.g. if your CSV has no header, adjust `names=` above

    # 2) Parse date and keep just the date part
    df[date_col] = pd.to_datetime(df[date_col]).dt.date

    today = datetime.date.today()

    reports = []
    # 3) For each city
    for city, grp in df.groupby(city_col):

        last_date = grp[date_col].max()
        if last_date < today:
            # 4) Missing range: day after last_date up to yesterday
            start_missing = last_date + datetime.timedelta(days=1)
            end_missing   = today #- datetime.timedelta(days=1)
            reports.append(
                f"{city} is missing data from {start_missing.isoformat()} to {end_missing.isoformat()}"
            )
        else:
            reports.append(f"{city} is up-to-date (has {last_date.isoformat()})")
    return reports



In [50]:
input_path = "../data/processed/weather_energy_data.csv"
response = report_city_freshness(input_path)

In [51]:
print(response)

['Chicago is missing data from 2025-08-03 to 2025-08-05', 'Houston is missing data from 2025-08-04 to 2025-08-05', 'New York is missing data from 2025-08-03 to 2025-08-05', 'Phoenix is missing data from 2025-08-04 to 2025-08-05', 'Seattle is missing data from 2025-08-04 to 2025-08-05']


trying to display geographic results 

In [16]:
import yaml
import pandas as pd
with open("../config/config.yaml", "r") as file:
    config = yaml.safe_load(file)
    
cities = config["cities"]
df_cities = pd.DataFrame(cities)


In [12]:
df_cities["Latitude"] = df_cities["Latitude"].astype(float)

In [17]:
print(df_cities.dtypes)

city          object
state         object
station       object
region        object
timezone      object
Latitude     float64
Longitude    float64
dtype: object


In [4]:
df_cities.head()

,city,state,station,region,timezone,Latitude,Longitude
0,New York,New York,GHCND:USW00094728,NYIS,Eastern,40.7128,-74.0060
1,Chicago,Illinois,GHCND:USW00094846,PJM,Eastern,41.8781,-87.6298
2,Houston,Texas,GHCND:USW00012960,ERCO,Central,29.7604,-95.3698
3,Phoenix,Arizona,GHCND:USW00023183,AZPS,Mountain,33.4484,-112.0740
4,Seattle,Washington,GHCND:USW00024233,SCL,Pacific,47.6062,-122.3321


In [7]:
df_2 = df_cities[["city", "state", "Latitude", "Longitude"]].copy()
df_2.head()

,city,state,Latitude,Longitude
0,New York,New York,40.7128,-74.0060
1,Chicago,Illinois,41.8781,-87.6298
2,Houston,Texas,29.7604,-95.3698
3,Phoenix,Arizona,33.4484,-112.0740
4,Seattle,Washington,47.6062,-122.3321


In [8]:
df = pd.read_csv("../data/processed/weather_energy_data.csv")
df.head(2)

,date,city,state,TMAX,TMIN,respondent-name,timezone,value-units,Demand,Net generation
0,2025-03-01,Chicago,Illinois,34.0,22.0,"PJM Interconnection, LLC",Eastern,megawatthours,2110785.0,2206107.0
1,2025-03-01,Houston,Texas,85.0,51.0,"Electric Reliability Council of Texas, Inc.",Central,megawatthours,1059205.0,1057815.0


In [9]:
# Merge on date, city, and state
merged_df = pd.merge(df, df_2, on=["city", "state"], how="inner")


In [31]:
merged_df.tail()

,date,city,state,TMAX,TMIN,respondent-name,timezone,value-units,Demand,Net generation,Latitude,Longitude
774,2025-08-04,Phoenix,Arizona,112.0,87.0,Arizona Public Service Company,Mountain,megawatthours,152011.0,79127.0,33.4484,-112.0740
775,2025-08-04,Seattle,Washington,71.0,58.0,Seattle City Light,Pacific,megawatthours,23601.0,8447.0,47.6062,-122.3321
776,2025-08-05,Houston,Texas,96.0,77.0,"Electric Reliability Council of Texas, Inc.",Central,megawatthours,1604201.0,1585017.0,29.7604,-95.3698
777,2025-08-05,Phoenix,Arizona,114.0,90.0,Arizona Public Service Company,Mountain,megawatthours,161623.0,85705.0,33.4484,-112.0740
778,2025-08-05,Seattle,Washington,78.0,57.0,Seattle City Light,Pacific,megawatthours,23678.0,7600.0,47.6062,-122.3321


In [32]:
def get_pct_change(df):
    df.sort_values(by=["city", "date"])

    # Group by city and shift Demand column
    df["Demand_yesterday"] = df.groupby("city")["Demand"].shift(1)

    # Calculate percentage change
    df["Demand_pct_change"] = round(((df["Demand"] - df["Demand_yesterday"]) / df["Demand_yesterday"]) * 100, 2)
    return df


In [33]:
response = get_pct_change(merged_df)

In [36]:
response.head(20)

,date,city,state,TMAX,TMIN,respondent-name,timezone,value-units,Demand,Net generation,Latitude,Longitude,Demand_yesterday,Demand_pct_change
0,2025-03-01,Chicago,Illinois,34.0,22.0,"PJM Interconnection, LLC",Eastern,megawatthours,2110785.0,2206107.0,41.8781,-87.6298,NaN,NaN
1,2025-03-01,Houston,Texas,85.0,51.0,"Electric Reliability Council of Texas, Inc.",Central,megawatthours,1059205.0,1057815.0,29.7604,-95.3698,NaN,NaN
2,2025-03-01,New York,New York,64.0,27.0,New York Independent System Operator,Eastern,megawatthours,391122.0,365188.0,40.7128,-74.0060,NaN,NaN
3,2025-03-01,Phoenix,Arizona,85.0,58.0,Arizona Public Service Company,Mountain,megawatthours,82316.0,58875.0,33.4484,-112.0740,NaN,NaN
4,2025-03-01,Seattle,Washington,62.0,37.0,Seattle City Light,Pacific,megawatthours,25460.0,9262.0,47.6062,-122.3321,NaN,NaN
5,2025-03-02,Chicago,Illinois,37.0,18.0,"PJM Interconnection, LLC",Eastern,megawatthours,2360071.0,2402257.0,41.8781,-87.6298,2110785.0,11.81
6,2025-03-02,Houston,Texas,74.0,55.0,"Electric Reliability Council of Texas, Inc.",Central,megawatthours,1051763.0,1049151.0,29.7604,-95.3698,1059205.0,-0.70
7,2025-03-02,New York,New York,33.0,20.0,New York Independent System Operator,Eastern,megawatthours,417602.0,381132.0,40.7128,-74.0060,391122.0,6.77
8,2025-03-02,Phoenix,Arizona,80.0,55.0,Arizona Public Service Company,Mountain,megawatthours,79043.0,60650.0,33.4484,-112.0740,82316.0,-3.98
9,2025-03-02,Seattle,Washington,51.0,44.0,Seattle City Light,Pacific,megawatthours,26038.0,9428.0,47.6062,-122.3321,25460.0,2.27
